In [75]:
import pandas as pd

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [76]:
CLIENT_ID = 'ANUWSG3MWQZFBJMCNASMW1AZXAUKFBD4GVZNJOGNM4N3SSNR' 
CLIENT_SECRET = 'NSCTOJNLJFXVCAYFJOIM01LIGH0HSGOPJ2NUIUBEZL12ZAIM' 
VERSION = '20180604'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: ANUWSG3MWQZFBJMCNASMW1AZXAUKFBD4GVZNJOGNM4N3SSNR
CLIENT_SECRET:NSCTOJNLJFXVCAYFJOIM01LIGH0HSGOPJ2NUIUBEZL12ZAIM


In [77]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of New York are 40.7127281, -74.0060152.


In [78]:
search_query = 'museum'
radius = 500

In [79]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ANUWSG3MWQZFBJMCNASMW1AZXAUKFBD4GVZNJOGNM4N3SSNR&client_secret=NSCTOJNLJFXVCAYFJOIM01LIGH0HSGOPJ2NUIUBEZL12ZAIM&ll=40.7127281,-74.0060152&v=20180604&query=museum&radius=500&limit=30'

In [80]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edbc7b571c428001b530b17'},
 'response': {'venues': [{'id': '4ab67030f964a520187720e3',
    'name': 'South Street Seaport Museum',
    'location': {'address': '12 Fulton St',
     'crossStreet': 'btwn Water and South St',
     'lat': 40.70720445017337,
     'lng': -74.00372874071756,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.70720445017337,
       'lng': -74.00372874071756},
      {'label': 'entrance', 'lat': 40.706641, 'lng': -74.003656}],
     'distance': 644,
     'postalCode': '10038',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['12 Fulton St (btwn Water and South St)',
      'New York, NY 10038',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d190941735',
      'name': 'History Museum',
      'pluralName': 'History Museums',
      'shortName': 'History Museum',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_

In [81]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",False,4ab67030f964a520187720e3,12 Fulton St,US,New York,United States,btwn Water and South St,644,"[12 Fulton St (btwn Water and South St), New Y...","[{'label': 'display', 'lat': 40.70720445017337...",40.707204,-74.003729,NaN,10038,NY,South Street Seaport Museum,v-1591462161,89016396
1,[],False,532dda20498e2530344bd327,NaN,US,New York,United States,NaN,576,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.717264, 'lng':...",40.717264,-74.002706,NaN,NaN,NY,Tiny Museum,v-1591462161,NaN
2,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",False,4beec8b4e24d20a1a8587314,City Hall Park,US,New York,United States,"original IRT Station, now abandoned",108,"[City Hall Park (original IRT Station, now aba...","[{'label': 'display', 'lat': 40.71278084415942...",40.712781,-74.004727,NaN,10007,NY,IRT Subway - City Hall (Abandoned),v-1591462161,65043378
3,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",False,4fbd8a35e4b09ec7cefcc578,4 Cortlandt Alley,US,New York,United States,btwn Franklin & White St,576,"[4 Cortlandt Alley (btwn Franklin & White St),...","[{'label': 'display', 'lat': 40.71733263447613...",40.717333,-74.002876,Downtown Manhattan,10013,NY,Mmuseumm,v-1591462161,NaN


In [82]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,South Street Seaport Museum,History Museum,12 Fulton St,US,New York,United States,btwn Water and South St,644,"[12 Fulton St (btwn Water and South St), New Y...","[{'label': 'display', 'lat': 40.70720445017337...",40.707204,-74.003729,NaN,10038,NY,4ab67030f964a520187720e3
1,Tiny Museum,None,NaN,US,New York,United States,NaN,576,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.717264, 'lng':...",40.717264,-74.002706,NaN,NaN,NY,532dda20498e2530344bd327
2,IRT Subway - City Hall (Abandoned),Museum,City Hall Park,US,New York,United States,"original IRT Station, now abandoned",108,"[City Hall Park (original IRT Station, now aba...","[{'label': 'display', 'lat': 40.71278084415942...",40.712781,-74.004727,NaN,10007,NY,4beec8b4e24d20a1a8587314
3,Mmuseumm,Museum,4 Cortlandt Alley,US,New York,United States,btwn Franklin & White St,576,"[4 Cortlandt Alley (btwn Franklin & White St),...","[{'label': 'display', 'lat': 40.71733263447613...",40.717333,-74.002876,Downtown Manhattan,10013,NY,4fbd8a35e4b09ec7cefcc578


The overall rating for Seaport Museum is obtained below

In [83]:
venue_id = '4ab67030f964a520187720e3' # ID of Seaport Museum
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4ab67030f964a520187720e3?client_id=ANUWSG3MWQZFBJMCNASMW1AZXAUKFBD4GVZNJOGNM4N3SSNR&client_secret=NSCTOJNLJFXVCAYFJOIM01LIGH0HSGOPJ2NUIUBEZL12ZAIM&v=20180604'

In [84]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ab67030f964a520187720e3',
 'name': 'South Street Seaport Museum',
 'contact': {'phone': '2127488786',
  'formattedPhone': '(212) 748-8786',
  'twitter': 'seaportmuseum',
  'facebook': '284637504913364',
  'facebookUsername': 'SouthStreetSeaportMuseum',
  'facebookName': 'South Street Seaport Museum'},
 'location': {'address': '12 Fulton St',
  'crossStreet': 'btwn Water and South St',
  'lat': 40.70720445017337,
  'lng': -74.00372874071756,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.70720445017337,
    'lng': -74.00372874071756},
   {'label': 'entrance', 'lat': 40.706641, 'lng': -74.003656}],
  'postalCode': '10038',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['12 Fulton St (btwn Water and South St)',
   'New York, NY 10038',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/south-street-seaport-museum/4ab67030f964a520187720e3',
 'categories': [{'id': '4bf58dd8d48988d190941735',
   'name'

In [85]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.2


The locations of the museums are visualised below

In [86]:
!pip install folium

In [87]:
import folium 

museums_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add a red circle marker to represent the centre of New York
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='City Centre',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(museums_map)

# add the museums as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(museums_map)

museums_map

The holiday maker can then refer to this map to find the best location to stay